In [21]:
import csv

def agg(m, price_point, sqft_point):
    m_stats = m.setdefault('stats', {})
    m_agg = m_stats.setdefault('total', 0)
    m_count = m_stats.setdefault('count', 0)
    m_dp = m_stats.setdefault('dp', [])
    m_sqft = m_stats.setdefault('sqft', 0)
    
    m_stats['total'] = m_agg + price_point
    m_stats['count'] = m_count + 1
    m_stats['dp'].append((price_point, sqft_point))
    m_stats['sqft'] = m_sqft + sqft_point
    
def calc(m):

    if type(m) != dict or 'stats' not in m:
        return
    m_stats = m['stats']
    m_stats['avg'] = m_stats['total'] / m_stats['count']
    
    if m_stats['sqft']:
        m_stats['avg_per_sqft'] = m_stats['total'] / m_stats['sqft']
    
    m_stats['dp'].sort()
    m_stats['median'] = m_stats['dp'][int(len(m_stats['dp']) * 0.5 - 1)]
    m_stats['p90'] = m_stats['dp'][int(len(m_stats['dp']) * 0.9 - 1)]

    
    
matrix = {}
with open('training.csv', 'r') as f_in:
    reader = csv.DictReader(f_in)
    i = 0
    for row in reader:
        
        # skip empty cities, property types, and prices
        if not row['CITY'] or not row['PROPERTY_TYPE'] or not row['PRICE'] or not row['SQUARE_FEET']:
            continue
        
        i += 1
        price_point = int(row['PRICE'])
        sqft_point = int(row['SQUARE_FEET'])
        city_level = matrix.setdefault(row['CITY'], {})
        p_type_level = city_level.setdefault(row['PROPERTY_TYPE'], {})
        bb_level = p_type_level.setdefault(str(row['BEDS']) + "_" + str(row['BATHS']), {})
        
        agg(city_level, price_point, sqft_point)
        agg(p_type_level, price_point, sqft_point)
        agg(bb_level, price_point, sqft_point)

    print("Used {} data points".format(i))
    
    for city in matrix:
        calc(matrix[city])
        for p_type in matrix[city]:
            calc(matrix[city][p_type])
            for bb in matrix[city][p_type]:
                calc(matrix[city][p_type][bb])    

Used 10367 data points


In [25]:
def query(in_city, # required
          in_p_type=None,  # optional
          n_beds=None,  # optional
          n_baths=None # optional
         ):
    
    in_city = in_city.upper().strip()
    if in_city not in matrix:
        raise Exception("City {} not in available cities: {}".format(in_city, matrix.keys()))
    city = matrix[in_city]
    
    if not in_p_type:
        return city['stats']
    
    in_p_type = in_p_type.upper().strip()
    if in_p_type not in matrix[in_city]:
        raise Exception("Property type {} not in available ptypes: {}".format(in_p_type, matrix[in_city].keys()))
        
    if not n_beds or not n_baths:
        return matrix[in_city][in_p_type]['stats']
    
    return matrix[in_city][in_p_type].get(str(n_beds) + "_" + str(float(n_baths)), "No data exists for {} beds and {} baths. Available: {}".format(n_beds, n_baths, matrix[in_city][in_p_type].keys()))
    
    
query(in_city='palo alto', in_p_type='single_family_home')




{'total': 1403764389,
 'count': 338,
 'dp': [(1215000, 1951),
  (1500000, 2553),
  (1605000, 636),
  (1650000, 760),
  (1700000, 1234),
  (1780000, 1208),
  (1850000, 760),
  (2000000, 1068),
  (2000000, 1478),
  (2080000, 1394),
  (2100000, 1316),
  (2100000, 1948),
  (2155000, 1656),
  (2200000, 1193),
  (2200000, 1909),
  (2220000, 1249),
  (2248000, 1912),
  (2250000, 1000),
  (2250000, 1043),
  (2300000, 1140),
  (2300000, 1268),
  (2300000, 1288),
  (2300000, 1754),
  (2325000, 900),
  (2330000, 876),
  (2345000, 1566),
  (2350000, 480),
  (2350000, 1008),
  (2370000, 2087),
  (2375000, 1848),
  (2400000, 1186),
  (2415000, 1080),
  (2420000, 1467),
  (2430000, 3973),
  (2452000, 1756),
  (2460000, 1414),
  (2468000, 1123),
  (2499000, 2513),
  (2500000, 992),
  (2500000, 1618),
  (2518000, 1475),
  (2525000, 1604),
  (2550000, 996),
  (2580000, 2424),
  (2600000, 1040),
  (2600000, 1530),
  (2600000, 1882),
  (2610000, 2241),
  (2630000, 1472),
  (2650000, 1044),
  (2675000, 176